In [1]:
import warnings
import tempfile

import numpy as np
from specutils import Spectrum1D
from astroquery.mast import Observations
from inspect import isclass
from glue.core import BaseData
from glue.core.subset import Subset, RangeSubsetState, RoiSubsetState
from glue.core.roi import XRangeROI
from glue.config import data_translator
from glue.core.edit_subset_mode import OrMode





from regions import PixCoord, CirclePixelRegion

from jdaviz import Specviz

specviz = Specviz()

data_dir = tempfile.gettempdir()

fn = "jw02732-o004_t004_miri_ch1-shortmediumlong_x1d.fits"

result = Observations.download_file(f"mast:JWST/product/{fn}", local_path=f'{data_dir}/{fn}')




with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    specviz.load_spectrum(f'{data_dir}/{fn}', "spec1")
    specviz.load_spectrum(f'{data_dir}/{fn}', "spec2")


specviz.show()

specviz.app.get_viewer("spectrum-viewer").apply_roi(XRangeROI(5.4, 5.6))
specviz.app.session.edit_subset_mode.mode = OrMode

specviz.app.get_viewer("spectrum-viewer").apply_roi(XRangeROI(6, 6.6))
specviz.app.get_viewer("spectrum-viewer").apply_roi(XRangeROI(7, 7.2))



INFO: Found cached file /var/folders/nm/dt67twk14fb68133q71wnrwh0001rs/T/jw02732-o004_t004_miri_ch1-shortmediumlong_x1d.fits with expected size 590400. [astroquery.query]


Application(config='specviz', events=['call_viewer_method', 'close_snackbar_message', 'data_item_remove', 'dat…

In [4]:
def get_select_data_from_viewer(
    # supply the helper:
    specviz,
    # 
    viewer_reference, 
    # if provided, only return associated data
    data_label=None, 
    # 
    cls='default',
    # whether to include subsets in the returned dictionary
    include_subsets=True,
    # whether to only include subsets in the returned dictionary
    only_subsets=False,
    # return subsets the old way i.e. as a mask applied to a spectrum1d object
    apply_subsets_to_data=False
                               
                               
):
    """
    
    """
    
    viewer = specviz.app.get_viewer(viewer_reference)
    cls = viewer.default_class if cls == 'default' else cls

    # TODO: Fix second part of the following statement
    if cls is not None and not isclass(cls):
        raise TypeError(
            "cls in get_data_from_viewer must be a class, None, or "
            "the 'default' string.")

    data = {}

    # If the viewer also supports collapsing, then grab the user's chosen
    #  statistic for collapsing data
    if hasattr(viewer.state, 'function'):
        statistic = viewer.state.function
    else:
        statistic = None

    for layer_state in viewer.state.layers:
        label = layer_state.layer.label

        if (hasattr(layer_state, 'layer') and
                (data_label is None or label == data_label)):

            # For raw data, just include the data itself
            if isinstance(layer_state.layer, BaseData):
                layer_data = layer_state.layer

                if cls is not None:
                    # If data is one-dimensional, assume that it can be
                    #  collapsed via the defined statistic
                    if 'Trace' in layer_data.meta:
                        layer_data = layer_data.get_object()
                    elif cls == Spectrum1D:
                        layer_data = layer_data.get_object(cls=cls,
                                                           statistic=statistic)
                    else:
                        layer_data = layer_data.get_object(cls=cls)
                # If the shape of the data is 2d, then use CCDData as the
                #  output data type
                elif len(layer_data.shape) == 2:
                    layer_data = layer_data.get_object(cls=CCDData)

                data[label] = layer_data

            # For subsets, make sure to apply the subset mask to the
            #  layer data first
            elif isinstance(layer_state.layer, Subset):
                if include_subsets:
#                     layer_data = layer_state.layer

#                     if cls is not None:
#                         handler, _ = data_translator.get_handler_for(cls)
#                         try:
#                             layer_data = handler.to_object(layer_data,
#                                                            statistic=statistic)
#                         except IncompatibleAttribute:
#                             continue

                    # data[label] = traverse(layer_state.layer.subset_state)
                    data[label] = layer_state.layer.subset_state.traverse()



    """
    how do we want subset information returned?
        - as before (applied to each data and returned as a masked spectrum1d)
            - apply subsets keyword in get_data takes list of strings that are subset names and apply those subsets to the data
        - as an astropy region showing all bounds of subset, even if compound?
        - as a glue subset object (not sure how beneficial this would be)
        - something else? 
            - define mask component for spectrum
            - do not want spectrum attaached to one particular mask
            - return as just a mask so that a user can use them as they wish in the notebook +
            - remove subsets from this method completely +
            - remove `get_viewer` from get subsets method (cubeviz case is special since a subset is spatial and also applied in spectrum-viewer as a collapsed function)
            - implement get_data_... at base helper class level so it can be public api
            - force get_data to require a data_label so data is returned along with associated subsets. This avoids returning as a dictionary where keys can be overwritten
            - get_data needs to know if data needs to be collapsed or not (use cls parameter, or overload cubeviz helper to have extra keyword arguement)
        - method for accessing data labels present/visible in jdaviz viewers
            
            
    
    problem with 1st option is explored here: https://github.com/spacetelescope/jdaviz/issues/1921

    problem with 2nd option is glue astronomy translators would need to be updated to handle 1d
    data and units would need to be assumed (glue subsets do not have units).
    """


    # If a data label was provided, return only the corresponding data, otherwise return all:
    return data.get(data_label, data)

In [1]:
import numpy as np
from astropy import units as u
from astropy.nddata import CCDData, StdDevUncertainty
from glue.core.edit_subset_mode import (AndMode, AndNotMode, OrMode,
                                        ReplaceMode, XorMode, NewMode)
import warnings

from specutils import Spectrum1D
from glue.core.roi import XRangeROI
from glue.config import data_translator

from jdaviz import Specviz

SPECTRUM_SIZE = 11
np.random.seed(42)

spec_axis = np.linspace(6000, 8000, SPECTRUM_SIZE) * u.AA
flux = (np.random.randn(len(spec_axis.value)) +
        10*np.exp(-0.001*(spec_axis.value-6563)**2) +
        spec_axis.value/500) * u.Jy
uncertainty = StdDevUncertainty(np.abs(np.random.randn(len(spec_axis.value))) * u.Jy)

meta = dict(header=dict(FILENAME="jdaviz-test-file.fits"))

s = Spectrum1D(spectral_axis=spec_axis, flux=flux, uncertainty=uncertainty, meta=meta)
s2 = Spectrum1D(spectral_axis=spec_axis+1000*u.AA, flux=flux, uncertainty=uncertainty, meta=meta)
print(s.spectral_axis)
print(s2.spectral_axis)


specviz = Specviz()

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    specviz.load_spectrum(s, "spec1")
    specviz.load_spectrum(s2, "spec2")

specviz.show()

specviz.app.get_viewer("spectrum-viewer").apply_roi(XRangeROI(6000, 6500))
specviz.app.session.edit_subset_mode.mode = NewMode

specviz.app.get_viewer("spectrum-viewer").apply_roi(XRangeROI(6700, 7200))
specviz.app.get_viewer("spectrum-viewer").apply_roi(XRangeROI(8200, 8800))

[6000. 6200. 6400. 6600. 6800. 7000. 7200. 7400. 7600. 7800. 8000.] Angstrom
[7000. 7200. 7400. 7600. 7800. 8000. 8200. 8400. 8600. 8800. 9000.] Angstrom


Application(config='specviz', events=['call_viewer_method', 'close_snackbar_message', 'data_item_remove', 'dat…

In [2]:


def get_data(specviz, data_label, cls='default', subsets_to_apply=None, statistic=None):
    """
    Returns data of the name data_label of type cls with subsets applied from
    subsets_to_apply.
    
    Parameters
    ----------
    viewer_reference : str
        The reference to the viewer defined with the ``reference`` key
        in the yaml configuration file.
    data_label : str
        Provide a label to retrieve a specific data set from the
        viewer instance.
    subsets_to_apply : str, list, optional
        Subsets that are to be applied to data before it is returned. If a string
        is given, it will be turned into a list and looped through. A future implementation
        will allow applying multiple subsets to data before it is returned.
    statistic : {'minimum', 'maximum', 'mean', 'median', 'sum', 'percentile'}, optional
        The statistic to use to collapse the dataset

    Returns
    -------
    data : dict
        A dict of the transformed Glue subset objects, with keys
        representing the subset name and values as astropy regions
        objects.
    
    """
    # Previously, this would use viewer.default_class. Do we want to pass
    # a viewer reference in order to enable this? Or move this to the base
    # viewer class?
    cls = Spectrum1D if cls == 'default' else cls
    data = None
    
    # Loop through each subset
    for subsets in specviz.app.data_collection.subset_groups:
        # If name matches name from the list subsets_to_apply, continue
        if subsets.label in subsets_to_apply:
            # Loop through each data an individual subset applies to
            for subset in subsets.subsets:
                # If the subset applies to data with the same name as data_label, continue
                if subset.data.label == data_label:
                    # print(subsets.label, subset.data.label)
                    # data = subset
                    # print(subset.data.get_mask(subset.subset_state))

                    # TODO: If applying multiple subsets to data, convert to
                    # mask array and then combine them together using an operator. Then,
                    # apply to data.
                    # print(np.where(subset.data.get_mask(subset.subset_state) == True))

                    if cls is not None:
                        handler, _ = data_translator.get_handler_for(cls)
                        try:
                            data = handler.to_object(subset, statistic=statistic)
                        except Exception as e:
                            print(e)
                            continue
    return data

In [4]:
results = specviz.get_data(data_label="spec1", subsets_to_apply=['Subset 1'], statistic=None)
# results2 = get_data(specviz, "spec1", Spectrum1D, ['Subset 1'])
print(results)
print(results.mask)
# specviz.load_spectrum(results, "subset_1_applied"
results.flux

Spectrum1D (length=11)
flux:             [ 12.497 Jy, ..., 15.537 Jy ],  mean=14.596 Jy
spectral axis:    [ 6000.0 Angstrom, ..., 8000.0 Angstrom ],  mean=7000.0 Angstrom
uncertainty:      [ StdDevUncertainty(0.46572975), ..., StdDevUncertainty(0.2257763) ]
[False False False  True  True  True  True  True  True  True  True]


<Quantity [12.49671415, 12.2617357 , 13.44768854, 17.26664179, 13.36584663,
           13.76586304, 15.97921282, 15.56743473, 14.73052561, 16.14256004,
           15.53658231] Jy>

In [7]:
s3 = s2._copy(spectral_axis=s2.spectral_axis+1000*u.AA)

In [23]:
a = [False, False, True]
# b = [True, False, False]
b = []
print([ai or bi for ai,bi in zip(a,b)])

[]


In [11]:
viewer = specviz.app.get_viewer("spectrum-viewer")

cls = viewer.default_class if cls == 'default' else cls
if hasattr(viewer.state, 'function'):
    statistic = viewer.state.function
else:
    statistic = None
data = None

for layer_state in viewer.state.layers:
    label = layer_state.layer.label
    # For subsets, make sure to apply the subset mask to the
    #  layer data first
    if isinstance(layer_state.layer, Subset):
        data = layer_state.layer
        print(type(data))
        if cls is not None:
            handler, _ = data_translator.get_handler_for(cls)
            try:
                data = handler.to_object(data, statistic=statistic)
            except Exception as e:
                print(e)
                continue
        print(data)
        # TODO: FIX
        print(np.where(data.mask == False))
        print(data.spectral_axis[625])

NameError: name 'cls' is not defined

In [26]:
# for key, value in specviz.app.get_data_from_viewer(specviz._default_spectrum_viewer_reference_name).items():
#     print(key, value, "\n\n")
print(specviz.app.get_data_from_viewer(specviz._default_spectrum_viewer_reference_name))
print("\n\n\n\n\n\n")
data = get_select_data_from_viewer(specviz, specviz._default_spectrum_viewer_reference_name, data_label=None)
data

{'spec1': <Spectrum1D(flux=<Quantity [84.28579203, 86.96449557, 91.65531698, ..., 87.60813172,
           86.21068459, 85.08942663] MJy / sr>, spectral_axis=<SpectralAxis [4.9004001 , 4.9012001 , 4.9020001 , ..., 7.64840003, 7.64920003,
   7.65000003] um>, uncertainty=StdDevUncertainty([0.22357993, 0.2183111 , 0.22367723, ..., 0.21385001,
                   0.20442478, 0.20699027]))>, 'spec2': <Spectrum1D(flux=<Quantity [84.28579203, 86.96449557, 91.65531698, ..., 87.60813172,
           86.21068459, 85.08942663] MJy / sr>, spectral_axis=<SpectralAxis [4.9004001 , 4.9012001 , 4.9020001 , ..., 7.64840003, 7.64920003,
   7.65000003] um>, uncertainty=StdDevUncertainty([0.22357993, 0.2183111 , 0.22367723, ..., 0.21385001,
                   0.20442478, 0.20699027]))>, 'Subset 1': <Spectrum1D(flux=<Quantity [84.28579203, 86.96449557, 91.65531698, ..., 87.60813172,
           86.21068459, 85.08942663] MJy / sr>, spectral_axis=<SpectralAxis [4.9004001 , 4.9012001 , 4.9020001 , ..., 7.64840003

{'spec1': <Spectrum1D(flux=<Quantity [84.28579203, 86.96449557, 91.65531698, ..., 87.60813172,
            86.21068459, 85.08942663] MJy / sr>, spectral_axis=<SpectralAxis [4.9004001 , 4.9012001 , 4.9020001 , ..., 7.64840003, 7.64920003,
    7.65000003] um>, uncertainty=StdDevUncertainty([0.22357993, 0.2183111 , 0.22367723, ..., 0.21385001,
                    0.20442478, 0.20699027]))>,
 'spec2': <Spectrum1D(flux=<Quantity [84.28579203, 86.96449557, 91.65531698, ..., 87.60813172,
            86.21068459, 85.08942663] MJy / sr>, spectral_axis=<SpectralAxis [4.9004001 , 4.9012001 , 4.9020001 , ..., 7.64840003, 7.64920003,
    7.65000003] um>, uncertainty=StdDevUncertainty([0.22357993, 0.2183111 , 0.22367723, ..., 0.21385001,
                    0.20442478, 0.20699027]))>,
 'Subset 1': (<function _operator.or_(a, b, /)>,
  (6.260256582743592, 6.392613101139116),
  (<function _operator.or_(a, b, /)>,
   (6.260256582743592, 6.392613101139116),
   (<function _operator.or_(a, b, /)>,
    (6.

<br>
Subset 1 is kept as a SubsetState object and traversed to see all of it's subregions. This is different to how things are done in main currently, where Subset 1 is applied to a spectrum1d object and returned as such. You can see that no units are provided because glue subsets do not have units by default, only when applied to a spectrum1d object. We would need to assume units for the subsets in order to return them as astropy regions objects, as shown below.

In [27]:
specviz.get_spectral_regions()

{'Subset 1': Spectral Region, 4 sub-regions:
   (5.400400082726264 um, 5.599600077694049 um)  
   (6.0004000675689895 um, 6.230000061768806 um) 
   (6.260400061000837 um, 6.599600052431924 um)  
   (7.000400042306865 um, 7.199600037274649 um)  }

In [18]:
specviz.app.data_collection.subset_groups[0].subsets


[Subset: Subset 1 (data: spec1), Subset: Subset 1 (data: spec2)]

In [23]:
    print(len(specviz.app.data_collection.subset_groups))
    
    for sg in specviz.app.data_collection.subset_groups:
        for sg_subset in sg.subsets:
            print(sg_subset.data.primary_components)
            print(sg_subset.data.pixel_component_ids)
            sg_subset.data.get_selection_definition(format='astropy-regions')

1
[Pixel Axis 0 [x], World 0, flux, uncertainty]
[Pixel Axis 0 [x]]
World 0


/Users/javerbukh/miniconda3/envs/test_this_f_s_env/lib/python3.9/site-packages/glue/core/data.py:2039: UserWarning: Data.primary_components is deprecated
  warnings.warn('Data.primary_components is deprecated', UserWarning)


NotImplementedError: Can only handle 2-d datasets

In [30]:
specviz.app.data_collection.subset_groups[0].subset_state.traverse()

# do something different for Rangesubset, ROIsubset, and compound subset


(<function _operator.or_(a, b, /)>,
 (<function _operator.or_(a, b, /)>,
  (<function _operator.or_(a, b, /)>,
   <glue.core.subset.RangeSubsetState at 0x7ff5e0078610>)))

In [4]:
 def traverse(self=specviz.app.data_collection.subset_groups[0].subset_state):
        return (self.op if not isinstance(self, RangeSubsetState) else (self.lo, self.hi),
                (self.state1.lo, self.state1.hi) if not hasattr(self.state1, 'traverse') else self.state1.traverse(),
                (self.state2.lo, self.state2.hi) if not hasattr(self.state2, 'traverse') else self.state2.traverse()
                )

In [7]:
traverse()

(<function _operator.or_(a, b, /)>,
 (7, 7.2),
 (<function _operator.or_(a, b, /)>,
  <glue.core.subset.RangeSubsetState at 0x7fb66182e550>))

In [52]:
subset1 = specviz.app.data_collection.subset_groups[0].subset_state
print(subset1.state1.lo, subset1.state1.hi)
print(subset1.state2.op)
print(subset1.state2.state1.lo, subset1.state2.state1.hi)
print(subset1.state2.state2.lo, subset1.state2.state2.hi)

6.772604395887553 7.129113082533561
<built-in function or_>
6.772604395887553 7.129113082533561
6 6.6


In [29]:
specviz.show()

Application(config='specviz', events=['call_viewer_method', 'close_snackbar_message', 'data_item_remove', 'dat…

In [30]:
data

{'spec1': <Spectrum1D(flux=<Quantity [84.28579203, 86.96449557, 91.65531698, ..., 87.60813172,
            86.21068459, 85.08942663] MJy / sr>, spectral_axis=<SpectralAxis [4.9004001 , 4.9012001 , 4.9020001 , ..., 7.64840003, 7.64920003,
    7.65000003] um>, uncertainty=StdDevUncertainty([0.22357993, 0.2183111 , 0.22367723, ..., 0.21385001,
                    0.20442478, 0.20699027]))>,
 'spec2': <Spectrum1D(flux=<Quantity [84.28579203, 86.96449557, 91.65531698, ..., 87.60813172,
            86.21068459, 85.08942663] MJy / sr>, spectral_axis=<SpectralAxis [4.9004001 , 4.9012001 , 4.9020001 , ..., 7.64840003, 7.64920003,
    7.65000003] um>, uncertainty=StdDevUncertainty([0.22357993, 0.2183111 , 0.22367723, ..., 0.21385001,
                    0.20442478, 0.20699027]))>,
 'Subset 1': (<function _operator.or_(a, b, /)>,
  (6.260256582743592, 6.392613101139116),
  (<function _operator.or_(a, b, /)>,
   (6.260256582743592, 6.392613101139116),
   (<function _operator.or_(a, b, /)>,
    (6.